In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# --- апертурный фильтр
def aperture_func(nx, f_scale, delta): 
    fx, fy = f_scale * np.asarray(np.meshgrid(np.linspace(-nx//2, nx//2-1, nx), np.linspace(-nx//2, nx//2-1, nx)))
    res = np.abs(np.sinc(delta*fx)*np.sinc(delta*fy))**2
    res[np.isnan(res)] = 0
    return res
    
# --- модуль |f|^11/3 
def abs_f(nx, f_scale):
    fx, fy = f_scale * np.asarray(np.meshgrid(np.linspace(-nx//2, nx//2-1, nx), np.linspace(-nx//2, nx//2-1, nx)))
    with np.errstate(divide='ignore'):
        res = pow(np.sqrt(fx**2+fy**2), -11./3.)
    
    res[np.isnan(res)] = 0
    return res

# --- фильтр Френеля
def ffilter(nx, f_scale, z, lambda_): 
    fx, fy = f_scale * np.asarray(np.meshgrid(np.linspace(-nx//2, nx//2-1, nx), np.linspace(-nx//2, nx//2-1, nx)))
    res = pow(np.sin(np.pi*z*lambda_*(fx**2+fy**2)), 2) / pow(lambda_, 2)
    res[np.isnan(res)] = 0
    return res

# --- монохроматические гаммы со скоростями 0 м/с
def gamma_mono(Cn2, z, lambda_, cjk=None, D=None, sec_per_frame=None, latency=None):
    const = 9.69*pow(10, -3)*16*pow(np.pi, 2)
    nx=cjk.shape[0]
    f_scale = 1/(2*D)
    delta = D/(cjk.shape[0]//2)

    Fresnel_filter = ffilter(nx, f_scale, z, lambda_)
    f_11_3 = abs_f(nx, f_scale)
    A_f = aperture_func(nx, f_scale, delta) 
    
    with np.errstate(invalid='ignore'):
        res = f_11_3 * Fresnel_filter
    
    res[np.isnan(res)] = 0
    res = res * A_f
    res[np.isnan(res)] = 0
    
    res = np.fft.fftshift(np.fft.irfft2(np.fft.fftshift(res), s=res.shape, norm='backward'))
    res = res * 9.69*pow(10, -3)*16*pow(np.pi, 2)

    res = res * pow(f_scale*nx, 2)
    res = Cn2 * res   
    
    res = res * cjk
    return res

In [5]:
# надо сравнить ее с резами Матевея
cjk = np.ones((452, 452))
D=2.5
latency=4
sec_per_frame=0.01
tmp = gamma_mono(1e-14, 2000, 650e-9, cjk=cjk, D=D, sec_per_frame=sec_per_frame, latency=latency)
print(np.max(tmp))

0.0014853766887197475


In [6]:
st = time.perf_counter()

a1 = np.linspace(0, 50000, 25)
gammas1 = np.ndarray(shape=(len(a1), cjk.shape[0], cjk.shape[1]))

for i in range(len(a1)):
    gammas1[i] = gamma_mono(1e-13, a1[i], 500e-9, cjk=cjk, D=D, sec_per_frame=sec_per_frame, latency=latency)
np.save('D:/astro/domecam/tempGM500.npy', gammas1)
print('temp gammas done!')
print('time:', time.perf_counter()-st)

gammas library done!
time: 1.2578088000000207
